# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
nltk.download()

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV


/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/enzo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package punkt to /home/enzo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/enzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('InsertTableName', con=engine)

relevant_categories = df.columns[4:]

X = df['message']
y = df[relevant_categories]

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X.head(5)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    # Replace any URL with  urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # remove punctuation and Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
        
    tokens = word_tokenize(text)
    
    # Remove stop-words
    stop_words = stopwords.words("english")
    tokens = [token for token in tokens if token not in stop_words]
    
    # lemmitize
    lemmatizer = nltk.WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
tokenize("testing: tested I do not doesn't lovVed loves loving you warnings .... http://www.hotmail.com")

['testing', 'tested', 'lovved', 'love', 'loving', 'warning', 'urlplaceholder']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LinearSVC(multi_class="crammer_singer"), n_jobs=1))
        #('clf', RandomForestClassifier())
    ])
def test_pipeline(pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    pipeline.fit(X_train, y_train)
    y_test_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_test_pred, target_names=relevant_categories))
test_pipeline(pipeline)   

                        precision    recall  f1-score   support

               related       0.87      0.91      0.89      4961
               request       0.75      0.61      0.67      1122
                 offer       0.00      0.00      0.00        31
           aid_related       0.73      0.71      0.72      2691
          medical_help       0.59      0.29      0.39       537
      medical_products       0.74      0.34      0.46       339
     search_and_rescue       0.61      0.14      0.23       176
              security       0.17      0.01      0.02       114
              military       0.56      0.33      0.42       204
                 water       0.69      0.67      0.68       386
                  food       0.79      0.77      0.78       726
               shelter       0.75      0.60      0.67       547
              clothing       0.72      0.55      0.62        97
                 money       0.57      0.27      0.36       160
        missing_people       0.80      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...rammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Evaluation for the train set 

In [9]:
y_train_pred = pipeline.predict(X_train)
print(classification_report(y_train, y_train_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.95      0.98      0.96     14928
               request       0.92      0.77      0.84      3332
                 offer       1.00      0.40      0.57        93
           aid_related       0.91      0.89      0.90      8135
          medical_help       0.92      0.63      0.75      1557
      medical_products       0.95      0.65      0.77       972
     search_and_rescue       0.98      0.52      0.68       547
              security       1.00      0.44      0.61       372
              military       0.97      0.88      0.92       654
                 water       0.94      0.88      0.91      1237
                  food       0.93      0.90      0.91      2186
               shelter       0.94      0.81      0.87      1740
              clothing       0.93      0.78      0.85       292
                 money       0.98      0.72      0.83       445
        missing_people       0.97      

#### Evaluation for the test set 

In [10]:
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.87      0.91      0.89      4978
               request       0.77      0.59      0.66      1142
                 offer       0.00      0.00      0.00        25
           aid_related       0.73      0.71      0.72      2725
          medical_help       0.60      0.31      0.41       527
      medical_products       0.67      0.35      0.46       341
     search_and_rescue       0.71      0.18      0.29       177
              security       0.36      0.04      0.07        99
              military       0.54      0.32      0.40       206
                 water       0.71      0.64      0.67       435
                  food       0.81      0.74      0.77       737
               shelter       0.74      0.61      0.67       574
              clothing       0.76      0.50      0.61       113
                 money       0.66      0.27      0.38       159
        missing_people       0.68      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegressionCV(multi_class="multinomial"), n_jobs=1))
        #('clf', RandomForestClassifier())
    ])
test_pipeline(pipeline) 

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4977
               request       0.96      0.12      0.21      1122
                 offer       0.00      0.00      0.00        30
           aid_related       0.89      0.25      0.39      2759
          medical_help       0.86      0.01      0.02       518
      medical_products       0.50      0.00      0.01       310
     search_and_rescue       1.00      0.01      0.01       182
              security       0.00      0.00      0.00       128
              military       0.00      0.00      0.00       226
                 water       0.77      0.03      0.05       393
                  food       0.90      0.09      0.16       738
               shelter       0.94      0.06      0.11       556
              clothing       1.00      0.04      0.08        94
                 money       0.00      0.00      0.00       165
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip... refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0),
           n_jobs=1))])

In [13]:
y_train_pred = pipeline.predict(X_train)
print(classification_report(y_train, y_train_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.93      0.98      0.96     14928
               request       0.97      0.12      0.22      3332
                 offer       0.00      0.00      0.00        93
           aid_related       0.94      0.27      0.42      8135
          medical_help       1.00      0.01      0.02      1557
      medical_products       0.88      0.01      0.01       972
     search_and_rescue       1.00      0.01      0.01       547
              security       0.00      0.00      0.00       372
              military       1.00      0.01      0.02       654
                 water       0.97      0.03      0.05      1237
                  food       0.94      0.07      0.14      2186
               shelter       0.98      0.05      0.10      1740
              clothing       1.00      0.01      0.03       292
                 money       0.00      0.00      0.00       445
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      4978
               request       0.90      0.15      0.26      1142
                 offer       0.00      0.00      0.00        25
           aid_related       0.87      0.27      0.41      2725
          medical_help       0.62      0.01      0.02       527
      medical_products       0.67      0.01      0.01       341
     search_and_rescue       0.00      0.00      0.00       177
              security       0.00      0.00      0.00        99
              military       1.00      0.02      0.05       206
                 water       0.68      0.03      0.07       435
                  food       0.85      0.08      0.14       737
               shelter       0.83      0.07      0.14       574
              clothing       0.00      0.00      0.00       113
                 money       0.00      0.00      0.00       159
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
        #('clf', RandomForestClassifier())
    ])


In [25]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))])

In [26]:
y_train_pred = pipeline.predict(X_train)
print(classification_report(y_train, y_train_pred, target_names=relevant_categories))


                        precision    recall  f1-score   support

               related       0.80      0.99      0.89     14928
               request       0.91      0.30      0.45      3332
                 offer       0.00      0.00      0.00        93
           aid_related       0.86      0.74      0.80      8135
          medical_help       0.63      0.01      0.02      1557
      medical_products       0.33      0.00      0.01       972
     search_and_rescue       0.00      0.00      0.00       547
              security       0.00      0.00      0.00       372
              military       0.40      0.00      0.01       654
                 water       0.14      0.00      0.00      1237
                  food       0.83      0.05      0.10      2186
               shelter       0.68      0.01      0.03      1740
              clothing       0.00      0.00      0.00       292
                 money       0.00      0.00      0.00       445
        missing_people       0.00      

In [27]:
y_test_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_test_pred, target_names=relevant_categories))

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      4978
               request       0.84      0.21      0.34      1142
                 offer       0.00      0.00      0.00        25
           aid_related       0.74      0.65      0.69      2725
          medical_help       0.00      0.00      0.00       527
      medical_products       0.00      0.00      0.00       341
     search_and_rescue       0.00      0.00      0.00       177
              security       0.00      0.00      0.00        99
              military       0.00      0.00      0.00       206
                 water       0.00      0.00      0.00       435
                  food       0.78      0.02      0.05       737
               shelter       0.50      0.00      0.00       574
              clothing       0.00      0.00      0.00       113
                 money       0.00      0.00      0.00       159
        missing_people       0.00      

/home/enzo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'vect__min_df': [1, 5, 10],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25, 50], 
              'clf__estimator__min_samples_split':[2, 5, 10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = 'f1_weighted', verbose = 1, cv=3)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
best_model_predict=cv.predict(X_test)

In [ ]:
print(classification_report(y_test, best_model_predict, target_names=relevant_categories))

In [ ]:
model_filepath = "andomForestClassifier.p"
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(cv, model_filepath)

print('Trained model saved!')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.